In [ ]:
import numpy as np 
import pandas as pd 
import os
import re
import math

In [ ]:
path = os.path.join(os.getcwd(),"Data")
stockspath = os.path.join(path,"Stock")
print(stockspath)
print(path)

In [ ]:
ind = pd.read_csv(os.path.join(path,"Index.csv"))

In [ ]:
ind["% Return"] = ((ind["Close"] / ind['Close'].shift(1))-1)*100
ind["% YTD"] = ((ind.tail(1)['Close'].values[0]/ind["Close"])-1)*100
ind.to_csv(os.path.join(path,"mod"+"Index.csv"),index=None)

In [ ]:
def calculate_beta(stock):
    stock["% Return of Company"] = ((stock["Close Price"] / stock['Close Price'].shift(1))-1)*100
    ind = pd.read_csv(os.path.join(path,"modIndex.csv"))
    ind.rename(columns={'Close':'Close Price of SP500', '% Return':'% Return of SP500'}, inplace=True)
    ind.drop(['Open', 'High', 'Low', '% YTD'], axis = 1,inplace=True) 
    ind["Date"] = pd.to_datetime(ind["Date"])
    stock["Date"] = pd.to_datetime(stock["Date"])
    stock = pd.merge(stock, ind, on="Date", how = "left")

    sp500 = stock["% Return of SP500"]
    company = stock["% Return of Company"]
    results = list()
    for i in range(stock.shape[0]):
        # cov = np.cov(company[i:],sp500[i:])[0][1]
        cov = np.ma.cov(np.ma.masked_invalid(np.array(company[i:],sp500[i:])),rowvar=False)
        var = np.nanvar(sp500[i:])
        res = var/cov
        results.append(res)
    stock["Beta"] = results
    return stock

In [ ]:
def calculate_alpha(stock):
    stock["% YTD of Company"] = ((stock.tail(1)['Close Price'].values[0]/stock["Close Price"])-1)*100
    ind = pd.read_csv(os.path.join(path,"modIndex.csv"))
    ind.drop(['Open', 'High', 'Low', "Close", "% Return"], axis = 1,inplace=True) 
    ind.rename(columns={'% YTD':'% YTD of SP500'}, inplace=True)
    ind["Date"] = pd.to_datetime(ind["Date"])
    stock["Date"] = pd.to_datetime(stock["Date"])
    stock = pd.merge(stock, ind, on="Date", how = "left")
    stock["Alpha"] = stock["% YTD of Company"] - (stock["Rate"] + (stock["Beta"] * (stock["% YTD of SP500"] - stock["Rate"])))
    return stock

In [ ]:
def add_risk_free_column(stock):
    riskrates = pd.read_csv(os.path.join(path,"RiskFreeRate.csv"))
    riskrates["Date"] = pd.to_datetime(riskrates["Date"])
    stock["Date"] = pd.to_datetime(stock["Date"])
    riskrates["Rate"] = pd.to_numeric(riskrates["Rate"])
    res = pd.merge(stock, riskrates, on="Date", how = "left")
    return res

In [ ]:
for name in os.listdir(stockspath):
    stock = pd.read_csv(os.path.join(stockspath,name))
    stock = add_risk_free_column(stock)
    stock = calculate_beta(stock)
    stock = calculate_alpha(stock)
    stock.to_csv(os.path.join(stockspath,"ab"+name),index=None)